<a href="https://colab.research.google.com/github/totminaekaterina/RUSSE-2022-Detoxification/blob/main/detox_ruprompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ruprompts 

  Using cached torch-1.13.1-cp39-cp39-win_amd64.whl (162.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\totmi\\AppData\\Local\\Temp\\pip-uninstall-n8wvivke\\_c.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



## Training

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
print(torch.__version__)

2.0.0


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Load the dataset:

In [5]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={"train": "train_paraphrase_1_1_.csv", "validation": "dev_paraphrase_1_2_.csv"}, encoding='utf-8-sig', sep=';')
train_dataset = datasets["train"]
valid_dataset = datasets["validation"]

Found cached dataset csv (C:/Users/totmi/.cache/huggingface/datasets/csv/default-0aa63342a8b097a1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

Load the backbone:

In [6]:
from transformers import GPT2LMHeadModel, AutoTokenizer

backbone_id = "16_model_3"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Define the prompt format:

In [7]:
from ruprompts import PromptFormat

prompt_format = PromptFormat("<P*100>{Sentence_1}<P*20>")

Define the parametrization of trainable embeddings:

In [8]:
from ruprompts import TensorPromptProvider
from transformers import set_seed

set_seed(1)

prompt_provider = TensorPromptProvider()

Compose prompt format and prompt provider into prompt object and apply it to the model and tokenizer, i.e. add special tokens to the tokenizer and modify the layer of input embeddings of the model:

In [9]:
from ruprompts import Prompt

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

Preprocess the data:
1. format the data entries with the specified prompt format
2. tokenize the resulting sequences
3. truncate the `truncation_field` if sequence length exceeds `max_tokens`

In [10]:
from ruprompts import Text2TextPreprocessor

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="Sentence_2",
    max_tokens=1792,
    truncation_field="Sentence_1",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

Loading cached processed dataset at C:\Users\totmi\.cache\huggingface\datasets\csv\default-0aa63342a8b097a1\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-1486065019f0af5b.arrow
Loading cached processed dataset at C:\Users\totmi\.cache\huggingface\datasets\csv\default-0aa63342a8b097a1\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-bd580923b93758d0.arrow


Define training arguments:

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    eval_steps=5000,
    save_steps=10000,
    logging_steps=5000,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    max_steps=50000,
    report_to="tensorboard",
    logging_dir="logs",
    seed=1,
)

Choose optimization options:

In [15]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=200,
    num_training_steps=training_args.max_steps,
)

Define the callbacks and start training:

In [16]:
from transformers import Trainer
from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: Sentence_1, Sentence_2, id. If Sentence_1, Sentence_2, id are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 19999
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 50000
  Number of trainable parameters = 125321472
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
5000,1.274700,1.057721
10000,1.193400,1.055092
15000,1.178000,1.051057
20000,1.166400,1.039577
25000,1.160100,1.049180
30000,1.154200,1.045146
35000,1.148800,1.045466
40000,1.144900,1.041831
45000,1.141100,1.043108
50000,1.137600,1.039046


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: Sentence_1, Sentence_2, id. If Sentence_1, Sentence_2, id are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: Sentence_1, Sentence_2, id. If Sentence_1, Sentence_2, id are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 16
Saving model checkpoint to .\checkpoint-10000
Configuration saved in .\checkpoint-10000\config.json
Model weights saved in .\checkpoint-10000\pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: Sentence_1, Sent

TrainOutput(global_step=50000, training_loss=1.16990791015625, metrics={'train_runtime': 16896.1707, 'train_samples_per_second': 47.348, 'train_steps_per_second': 2.959, 'total_flos': 6.450507969652224e+16, 'train_loss': 1.16990791015625, 'epoch': 40.0})

## Inference

Load prompt from the last checkpoint:

In [17]:
from transformers import pipeline

prompt = Prompt.from_pretrained(f"checkpoint-50000")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

In [18]:
ppln({"Sentence_1": "Ублюдок, мать твою, а ну иди сюда"}, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Ублюдок, мать твою, иди сюда'}]

Run inference:

In [20]:
import pandas as pd

In [22]:
test = pd.read_csv("Train_Only_Sentence.csv", encoding='utf-8-sig', sep=';')

In [26]:
datasets = load_dataset("csv", data_files={"test": "Train_Only_Sentence.csv"}, encoding='utf-8-sig', sep=';')
test_dataset = datasets["test"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/totmi/.cache/huggingface/datasets/csv/default-c7bd505825cc574f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
from tqdm import tqdm
import transformers

transformers.logging.set_verbosity_error()

beam_count = 5

predictions = []
    
for i in tqdm(test_dataset["text"]):
    options = ppln(
        {"Sentence_1": i},
        do_sample=False,
        num_beams=beam_count,
        num_return_sequences=beam_count,
    )

    options = [i["generated_text"].replace("<pad>", "") for i in options]
    answer = sorted(options, key=len)[-1]  # get longest answer
    predictions.append(answer)

100%|██████████| 1009458/1009458 [71:01:54<00:00,  3.95it/s]   


In [32]:
pd.DataFrame({"answer": predictions})["answer"].apply(pd.Series).to_csv("test_ParaGen_True.csv", index=False)